<h3>Language modelling</h3>
<p>Konrad Przewłoka</p>

<h4>Imports</h4>

In [13]:
#!pip install torch
#!pip install transformers
#!pip install sacremoses
from pprint import pprint
import os
import random
import math
import collections
import matplotlib.pyplot as plt
from prettytable import PrettyTable
import torch
import transformers

<h4>Loading Geotrend distillbert polish model<h4>

In [41]:
tokenizer_distilbert = transformers.AutoTokenizer.from_pretrained("Geotrend/distilbert-base-pl-cased")
model_distilbert = transformers.AutoModelForMaskedLM.from_pretrained("Geotrend/distilbert-base-pl-cased")

<h4>Loading polish BERT uncased model<h4>

In [43]:
tokenizer_bert_uncased = transformers.AutoTokenizer.from_pretrained("dkleczek/bert-base-polish-uncased-v1")

model_bert_uncased = transformers.AutoModelForMaskedLM.from_pretrained("dkleczek/bert-base-polish-uncased-v1")

Some weights of the model checkpoint at dkleczek/bert-base-polish-uncased-v1 were not used when initializing BertForMaskedLM: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


<h4>Loading polish BERT cased model</h4>

In [44]:
tokenizer_bert_cased = transformers.AutoTokenizer.from_pretrained("dkleczek/bert-base-polish-cased-v1")

model_bert_cased = transformers.AutoModelForMaskedLM.from_pretrained("dkleczek/bert-base-polish-cased-v1")

Some weights of the model checkpoint at dkleczek/bert-base-polish-cased-v1 were not used when initializing BertForMaskedLM: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


<h4>Utility array of models</h4>

In [45]:
models = [
    {'name':"Distilled BERT polish", 'model':model_distilbert,'tokenizer':tokenizer_distilbert},
    {'name':"Polish BERT uncased", 'model':model_bert_uncased,'tokenizer':tokenizer_bert_uncased},
    {'name':"Polish BERT cased", 'model':model_bert_cased,'tokenizer':tokenizer_bert_cased}
]

<h4>Polish cases checking</h4>

In [47]:
def unmask_sentece(model,sentence,num_tokens):
    sentence = sentence.replace('[MASK]',model['tokenizer'].mask_token)
    inputs = model['tokenizer'](sentence, return_tensors="pt")
    mask_token_index = torch.where(inputs["input_ids"] == model['tokenizer'].mask_token_id)[1]
    token_logits = model['model'](**inputs).logits
    mask_token_logits = token_logits[0, mask_token_index, :]
    top_tokens = torch.topk(mask_token_logits, num_tokens, dim=1).indices[0].tolist()
    return top_tokens

example_sentences=[
    "Warszawa to największe [MASK].",
    "Marek bardzo długo szukał [MASK].",
    "Prezenty podobały się [MASK].",
    "Po wejściu na dach można obejrzeć [MASK].",
    "Tomasz postanowił pojechać do domu z [MASK].",
    "Anna od dawna marzyła o [MASK].",
    "Witaj [MASK]!"
]

for model in models:
    print(model['name'])
    for sentence in example_sentences:
        print(sentence)
        for token in unmask_sentece(model, sentence,3):
            print(sentence.replace(model['tokenizer'].mask_token,model['tokenizer'].decode([token])))

Distilled BERT polish
Warszawa to największe [MASK].
Warszawa to największe miasto.
Warszawa to największe miasta.
Warszawa to największe Miasto.
Marek bardzo długo szukał [MASK].
Marek bardzo długo szukał gry.
Marek bardzo długo szukał dzieci.
Marek bardzo długo szukał ojca.
Prezenty podobały się [MASK].
Prezenty podobały się LGBT.
Prezenty podobały się temat.
Prezenty podobały się tzw.
Po wejściu na dach można obejrzeć [MASK].
Po wejściu na dach można obejrzeć ulice.
Po wejściu na dach można obejrzeć miejsca.
Po wejściu na dach można obejrzeć budynek.
Tomasz postanowił pojechać do domu z [MASK].
Tomasz postanowił pojechać do domu z rodziny.
Tomasz postanowił pojechać do domu z domu.
Tomasz postanowił pojechać do domu z Warszawy.
Anna od dawna marzyła o [MASK].
Anna od dawna marzyła o wojnie.
Anna od dawna marzyła o romans.
Anna od dawna marzyła o tron.
Witaj [MASK]!
Witaj !!
Witaj ##sz!
Witaj Jak!
Polish BERT uncased
Warszawa to największe [MASK].
Warszawa to największe miasto.
Warsz

<h4>Testing long-range relationships</h4>

In [48]:
example_sentences_relationships=[
    "Anna wybrała się do sklepu, gdzie [MASK] mleko.",
    "Marek wrócił do domu i natychmiast [MASK] na kanapie.",
    "To drzewo [MASK] 50 lat zanim spłonęło."
]
for model in models:
    print(model['name'])
    for sentence in example_sentences_relationships:
        print(sentence)
        for token in unmask_sentece(model, sentence,2):
            print(sentence.replace(model['tokenizer'].mask_token,model['tokenizer'].decode([token])))

Distilled BERT polish
Anna wybrała się do sklepu, gdzie [MASK] mleko.
Anna wybrała się do sklepu, gdzie miała mleko.
Anna wybrała się do sklepu, gdzie zdobyła mleko.
Marek wrócił do domu i natychmiast [MASK] na kanapie.
Marek wrócił do domu i natychmiast zmarł na kanapie.
Marek wrócił do domu i natychmiast przeszedł na kanapie.
To drzewo [MASK] 50 lat zanim spłonęło.
To drzewo około 50 lat zanim spłonęło.
To drzewo było 50 lat zanim spłonęło.
Polish BERT uncased
Anna wybrała się do sklepu, gdzie [MASK] mleko.
Anna wybrała się do sklepu, gdzie kupuje mleko.
Anna wybrała się do sklepu, gdzie pija mleko.
Marek wrócił do domu i natychmiast [MASK] na kanapie.
Marek wrócił do domu i natychmiast spał na kanapie.
Marek wrócił do domu i natychmiast usiadł na kanapie.
To drzewo [MASK] 50 lat zanim spłonęło.
To drzewo miało 50 lat zanim spłonęło.
To drzewo przetrwało 50 lat zanim spłonęło.
Polish BERT cased
Anna wybrała się do sklepu, gdzie [MASK] mleko.
Anna wybrała się do sklepu, gdzie kupiła m

<h4>Checking real-world knolwedge</h4>

In [50]:
example_sentences_relationships=[
    "[MASK] wrze w temperaturze 100 stopni, a zamarza w temperaturze 0 stopni Celsjusza.",
    "Tomek studiuje polonistykę na [MASK].",
    "[MASK] wschodzi o ósmej i zachodzi około szesnastej."
]
for model in models:
    print(model['name'])
    for sentence in example_sentences_relationships:
        print(sentence)
        for token in unmask_sentece(model, sentence,2):
            print(sentence.replace(model['tokenizer'].mask_token,model['tokenizer'].decode([token])))

Distilled BERT polish
[MASK] wrze w temperaturze 100 stopni, a zamarza w temperaturze 0 stopni Celsjusza.
Na wrze w temperaturze 100 stopni, a zamarza w temperaturze 0 stopni Celsjusza.
We wrze w temperaturze 100 stopni, a zamarza w temperaturze 0 stopni Celsjusza.
Tomek studiuje polonistykę na [MASK].
Tomek studiuje polonistykę na Uniwersytecie.
Tomek studiuje polonistykę na polski.
[MASK] wschodzi o ósmej i zachodzi około szesnastej.
Od wschodzi o ósmej i zachodzi około szesnastej.
Grupa wschodzi o ósmej i zachodzi około szesnastej.
Polish BERT uncased
[MASK] wrze w temperaturze 100 stopni, a zamarza w temperaturze 0 stopni Celsjusza.
woda wrze w temperaturze 100 stopni, a zamarza w temperaturze 0 stopni Celsjusza.
nie wrze w temperaturze 100 stopni, a zamarza w temperaturze 0 stopni Celsjusza.
Tomek studiuje polonistykę na [MASK].
Tomek studiuje polonistykę na uniwersytecie.
Tomek studiuje polonistykę na harvardzie.
[MASK] wschodzi o ósmej i zachodzi około szesnastej.
i wschodzi o ó

<h4>Answers</h4>
<h5>I</h5>
<p>The model to produce best results was Polish BERT cased, available at: https://huggingface.co/dkleczek/bert-base-polish-cased-v1</p>
<h5>II</h5>
<p>All of tested models seem to capture at least a rudimentary level of polish grammar</p>
<h5>III</h5>
<p>All models display capability in terms of capturing long-distance relationships. That being said, the distilled BERT model results are of lower quality compared to others. The polish BERT cased model seems to offer the best results in terms of capturing long-distance relationships.</p>
<h5>IV</h5>
<p>Both polish BERT cased and uncased were able to capture significant level of real-world knowledge while the distilled BERT model was rather unsuccessful in this regard.</p>
<h5>V</h5>
<p>The most striking mistakes:
    <ul>
   <li> Na wrze w temperaturze 100 stopni, a zamarza w temperaturze 0 stopni Celsjusza.
   <li> We wrze w temperaturze 100 stopni, a zamarza w temperaturze 0 stopni Celsjusza.
   <li> Witaj !!
   <li> Witaj ##sz!
   <li> Prezenty podobały się tzw.
    </ul>
   Worst mistakes were made by the distilled BERT model.
</p>